### CG 

In [1]:
import numpy as np
import math
from scipy.sparse import diags

In [2]:
def generate_subroutine_bulk():
    ''' make 27pt stencil - bulk 3d '''
    outstring = ""
    outstring += "for( iz=1; iz< nz-1; iz++){\n" + \
          "    for( iy=1; iy< ny-1; iy++){\n" +\
          "        for (ix=1; ix < nx-1; ix++){\n" 
#    outstring += "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nlx+iz*nly*nlx]\n"
    outstring += "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nx+iz*ny*nx]\n"
    for i in range(-1,2):
        for j in range(-1,2):
            for k in range(-1,2):
                if i==j and j==k and i==0:
                    continue
                else:
#                    outstring+="- xv[ix+{} + (iy+{})*nlx + (iz+{})*nlx*nly]\n".format(i,j,k)
                    outstring+="- xv[ix+{} + (iy+{})*nx + (iz+{})*nx*ny]\n".format(i,j,k)
    outstring += ";}\n}\n}"
    
    #outstring.replace("+-","-")
    return outstring

In [3]:
from itertools import combinations

In [19]:
def zero_index_27pt():
    ''' corners '''
    def make_tempstring(l1,u1,l2,u2,l3,u3):
        #tempstring = "for (i{}=1; i{}<n{}-1;i{}++)".format(i2,i2,i2,i2) + "{\n"
        tempstring = "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nx+iz*ny*nx] \n"
        for n0 in range(l1,u1):
            for n1 in range(l2,u2):
                for n2 in range(l3,u3):
                    if(n0 == 0 and n1 ==n0 and n2==n1):
                        continue
                    tempstring += "-xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                    .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
        tempstring += ";\n"
        return tempstring
    i1,i2,i3 = 'x', 'y', 'z'
    outstring = ""

    def set_value(val, indx):
        if val == 0:
#            return "i{} = gi{}0 + 0;\n".format(indx,indx)
            return "i{} = 0;\n".format(indx,indx)
        elif val == 1:
#            return "i{} = gi{}0 + n{}-1;\n".format(indx,indx,indx)
            return "i{} = n{}-1;\n".format(indx,indx,indx)

    for ia in range(2):
        for ib in range(2):
            for ic in range(2):
                outstring += set_value(ia, 'x')
                outstring += set_value(ib, 'y')
                outstring += set_value(ic, 'z')
                outstring += make_tempstring(-ia,-ia+2,-ib,-ib+2,-ic,-ic+2)

    return outstring

In [23]:
def one_index_27pt():
    ''' edges '''
    outstring = "//edges \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l1,u1,l2,u2):
            tempstring = "for (i{}=1; i{}<n{}-1;i{}++)".format(i1,i1,i1,i1) + "{\n"
#            tempstring = "for (i{}t=1; i{}t<n{}-1;i{}t++)\n".format(i1,i1,i1,i1) + "{\n"
#            tempstring += "    i{} = i{}t + gi{}0;\n".format(i1,i1,i1)
#            tempstring += "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nlx+iz*nly*nlx] \n"
            tempstring += "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(l1,u1):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
#                        tempstring += "-xv[ix+iy*nlx+iz*nly*nlx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
#                        .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
                        tempstring += "-xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i1, "({}+{})".format("i"+i1,n0))\
                        .replace("i"+i2, "({}+{})".format("i"+i2,n1)).replace("i"+i3, "({}+{})".format("i"+i3,n2))
            tempstring += ";}\n"
            return tempstring
        outstring += "i{} = 0;\n".format(i2)
        outstring += "i{} = 0;\n".format(i3)
        outstring += make_tempstring(0,2,0,2)
        outstring += "i{} = 0;\n".format(i2)
        outstring += "i{} = n{}-1;\n".format(i3,i3)
        outstring += make_tempstring(0,2,-1,1)
        outstring += "i{} = n{}-1;\n".format(i2,i2)
        outstring += "i{} = 0;\n".format(i3)
        outstring += make_tempstring(-1,1,0,2)
        outstring += "i{} = n{}-1;\n".format(i2,i2)
        outstring += "i{} = n{}-1;\n".format(i3,i3)
        outstring += make_tempstring(-1,1,-1,1)
        
    return outstring

In [5]:
def two_index_27pt():
    ''' sides '''
    outstring = "//sides \n"
    dims = 'xyz'
    for c in combinations(dims, 2):
        i1,i2,i3 = [i for i in dims if i not in c][0],c[0], c[1]
        def make_tempstring(l2,u2):
            tempstring = "for (i{}=1; i{}<n{}-1;i{}++)".format(i2,i2,i2,i2) + "{\n"
            tempstring += "    for (i{}=1; i{}<n{}-1;i{}++)\n".format(i3,i3,i3,i3) + "{\n"
            tempstring += "yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nx+iz*ny*nx] \n"
            for n0 in range(-1,2):
                for n1 in range(-1,2):
                    for n2 in range(l2,u2):
                        if(n0 == 0 and n1 ==n0 and n2==n1):
                            continue
                        tempstring += "-xv[ix+iy*nx+iz*ny*nx]\n".replace("i"+i2, "({}+{})".format("i"+i2,n0))\
                        .replace("i"+i3, "({}+{})".format("i"+i3,n1)).replace("i"+i1, "({}+{})".format("i"+i1,n2))
            tempstring += ";}\n}\n"
            return tempstring
        outstring += "i{} = 0;\n".format(i1)
        outstring += make_tempstring(0,2)
        outstring += "i{} = n{}-1;\n".format(i1,i1)
        outstring += make_tempstring(-1,1)
        
    return outstring

In [7]:
def generate_subtroutine():
    return generate_subroutine_bulk() + "\n" \
         + two_index_27pt() + "\n" \
         + one_index_27pt() + "\n" \
         + zero_index_27pt()

In [26]:
with open('subroutine.cpp', 'w') as outfile:
    outfile.write(generate_subtroutine())
#print(generate_subtroutine())

In [25]:
print(generate_subtroutine())

for( iz=1; iz< nz-1; iz++){
    for( iy=1; iy< ny-1; iy++){
        for (ix=1; ix < nx-1; ix++){
yv[ix+iy*nx+iz*ny*nx] = 26. * xv[ix+iy*nx+iz*ny*nx]
- xv[ix+-1 + (iy+-1)*nx + (iz+-1)*nx*ny]
- xv[ix+-1 + (iy+-1)*nx + (iz+0)*nx*ny]
- xv[ix+-1 + (iy+-1)*nx + (iz+1)*nx*ny]
- xv[ix+-1 + (iy+0)*nx + (iz+-1)*nx*ny]
- xv[ix+-1 + (iy+0)*nx + (iz+0)*nx*ny]
- xv[ix+-1 + (iy+0)*nx + (iz+1)*nx*ny]
- xv[ix+-1 + (iy+1)*nx + (iz+-1)*nx*ny]
- xv[ix+-1 + (iy+1)*nx + (iz+0)*nx*ny]
- xv[ix+-1 + (iy+1)*nx + (iz+1)*nx*ny]
- xv[ix+0 + (iy+-1)*nx + (iz+-1)*nx*ny]
- xv[ix+0 + (iy+-1)*nx + (iz+0)*nx*ny]
- xv[ix+0 + (iy+-1)*nx + (iz+1)*nx*ny]
- xv[ix+0 + (iy+0)*nx + (iz+-1)*nx*ny]
- xv[ix+0 + (iy+0)*nx + (iz+1)*nx*ny]
- xv[ix+0 + (iy+1)*nx + (iz+-1)*nx*ny]
- xv[ix+0 + (iy+1)*nx + (iz+0)*nx*ny]
- xv[ix+0 + (iy+1)*nx + (iz+1)*nx*ny]
- xv[ix+1 + (iy+-1)*nx + (iz+-1)*nx*ny]
- xv[ix+1 + (iy+-1)*nx + (iz+0)*nx*ny]
- xv[ix+1 + (iy+-1)*nx + (iz+1)*nx*ny]
- xv[ix+1 + (iy+0)*nx + (iz+-1)*nx*ny]
- xv[ix+1 + (iy+0)*nx + (iz